In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import io
import numpy as np

In [2]:
URL = 'https://github.com/akutuzov/rushifteval_public/tree/main/durel/rushifteval1/data'

page = requests.get(URL)
  
soup = BeautifulSoup( page.content , 'html.parser')
find_by_class = soup.find_all('a', class_="Link--primary")

In [3]:
print(find_by_class,sep="\n")

[<a class="Link--primary markdown-title" data-pjax="true" data-test-selector="commit-tease-commit-message" href="/akutuzov/rushifteval_public/commit/9e1bda7c82939b920d5cab12cfda357c348ed7f6" title="Durel format">Durel format</a>, <a class="Link--primary text-bold" data-pjax="#repo-content-pjax-container" data-test-selector="commit-tease-commit-message" data-turbo-frame="repo-content-turbo-frame" href="/akutuzov/rushifteval_public/commit/9e1bda7c82939b920d5cab12cfda357c348ed7f6">Durel format</a>, <a class="pl-3 pr-3 py-3 p-md-0 mt-n3 mb-n3 mr-n3 m-md-0 Link--primary no-underline no-wrap" data-pjax="#repo-content-pjax-container" data-turbo-frame="repo-content-turbo-frame" href="/akutuzov/rushifteval_public/commits/main/durel/rushifteval1/data">
<svg aria-hidden="true" class="octicon octicon-history" data-view-component="true" height="16" text="gray" version="1.1" viewbox="0 0 16 16" width="16">
<path d="M1.643 3.143L.427 1.927A.25.25 0 000 2.104V5.75c0 .138.112.25.25.25h3.646a.25.25 0 00

In [4]:
judgements = []
uses = []
for i in find_by_class[3:]:
    judgements.append("https://raw.githubusercontent.com/"+i['href']+"/judgments.csv")
    uses.append("https://raw.githubusercontent.com/"+i['href']+"/uses.csv")

In [10]:
judgements_df = pd.DataFrame()
for i in judgements:
    judgements_df = pd.concat([judgements_df, pd.read_csv(io.StringIO(requests.get(i.replace("/tree","")).content.decode('utf-8')), delimiter='\t')])

In [11]:
uses_df = pd.DataFrame()
for i in uses:
    uses_df = pd.concat([uses_df, pd.read_csv(io.StringIO(requests.get(i.replace("/tree","")).content.decode('utf-8')), delimiter='\t')])

In [12]:
judgements_df.to_csv("judgements.csv", index=False)
uses_df.to_csv("uses.csv", index=False)

In [13]:
# Aggregate use pairs and extract median column
judgements_df = judgements_df.groupby(['identifier1', 'identifier2', 'lemma'])['judgment'].apply(list).reset_index(name='judgments')
#judgements_df['median_judgment'] = judgements_df['judgments'].apply(lambda x: np.nanmedian(list(x)))

# Remove pairs with nan median
#judgements_df = judgements_df[~judgements_df['median_judgment'].isnull()]
display(judgements_df)

,identifier1,identifier2,lemma,judgments
0,0,1,верховье,"[4.0, 3.0, 4.0]"
1,2,3,верховье,"[2.0, 3.0, 4.0]"
2,4,5,верховье,"[4.0, 4.0, 4.0]"
3,6,7,верховье,"[3.0, 2.0, 3.0]"
4,8,9,верховье,"[2.0, 2.0, 3.0]"
...,...,...,...,...
3298,6596,6597,ясли,"[4.0, 4.0, 4.0]"
3299,6598,6599,ясли,"[3.0, 1.0, 1.0]"
3300,6600,6601,ясли,"[4.0, 4.0, 4.0]"
3301,6602,6603,ясли,"[1.0, 3.0, 4.0]"


In [9]:
judgements_df.to_csv("judgements_grouped.csv", index=False)